In [8]:
import requests
from lxml import etree
import string
import time
import csv
import os


def get_tree(url):
    html = requests.get(url).text
    return etree.HTML(html)

def get_content(tree, xpath):
    result = []
    tree_node_list = tree.xpath(xpath)
    for node in tree_node_list:
        result.append(node.text)
    return result

def read_cve_csv(filepath):
    cve_list = []
    lines = open(filepath).readlines()
    for line in lines:
        line = line.strip('\r\n').split(',')
        if line[2]:
            cve_list.append(line[2])
    return list(set(cve_list))


def read_cves(file):
    cve_list = []
    with open(file) as f:
        data = csv.reader(f)
        for line in data:
           
            cve_list.append(line[2])
    return cve_list
    
def read_cve_list(filepath):
    cve_list = []
    lines = open(filepath).readlines()
    
    return [line.strip('\n').replace(' ','') for line in lines]
    
def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def process_cpe(tree,cpe_path):
    cpe_entry = get_content(tree, cpe_path)

    for item in cpe_entry:
        if item.startswith('cpe'):
            cpe = CPE(item)
            product = cpe.get_product()[0]
            version = cpe.get_version()[0]
            if not version or version.startswith('*') or version.startswith('-'):
                version = ''
                av_path = cpe_path[:-2]+'[2]/b/text()'
           
                av = tree.xpath(av_path)
                #print('-',av,'-')
                for v in av:
                    version += v+' '
               
    return product, version
          
#cveList = read_cve_list('../../result/seedKG/vulnerability.csv')
#cveList = read_cve_list('../../result/IE/cve_list.txt')
cveList = read_cves('../../result/IE/CERT/check_vuls.csv')
print(len(cveList))
from cpe import CPE
basepath = '//*[@id="p_lt_WebPartZone1_zoneCenter_pageplaceholder_p_lt_WebPartZone1_zoneCenter_VulnerabilityDetail_VulnFormView'
resultpath = '../../data/NVD/'
mkdir(resultpath)


result_cve_list = []
for filename in os.listdir(resultpath):
    result_cve_list.append(filename.replace('.txt',''))



1955
['cve-2017-12733', 'cve-2017-12731', 'cve-2016-0861', 'cve-2016-0862', 'cve-2017-2852', 'cve-2017-2853', 'cve-2017-2858', 'cve-2017-2860', 'cve-2017-2861', 'cve-2017-2867', 'cve-2017-2868', 'cve-2017-2869', 'cve-2017-9646', 'cve-2019-6557', 'cve-2019-6561', 'cve-2019-6565', 'cve-2019-6520', 'cve-2019-6524', 'cve-2019-6526', 'cve-2019-6522', 'cve-2019-6518', 'cve-2019-6563', 'cve-2019-6559', 'cve-2018-14825', '', 'cve-2018-8714', 'cve-2013-5944', 'cve-2017-9647', 'cve-2017-9633', 'cve-2014-5403', 'cve-2014-5405', 'cve-2014-5400', 'cve-2014-5401', 'cve-2015-3963', 'cve-2016-1488', 'cve-2016-3949', 'cve-2011-0340', 'cve-2014-0160', 'cve-2018-7529', 'cve-2018-7533', 'cve-2018-7531', 'cve-2019-7670', 'cve-2019-7669', 'cve-2019-7281', 'cve-2019-7280', 'cve-2019-7671', 'cve-2019-7667', 'cve-2019-7666', 'cve-2019-7672', 'cve-2019-9189', 'cve-2015-3977', 'cve-2012-2426', 'cve-2012-2427', 'cve-2012-2428', 'cve-2012-2429', 'cve-2016-0875', 'cve-2016-0876', 'cve-2016-0877', 'cve-2016-0878', '

In [ ]:

TEST = 0            
valid_nvd = 0     
        
if TEST:
    print("TEST")
    cveList = ['cve-2019-6557','cve-2019-16675','cve-2014-5408','cve-2016-5803','cve-2014-5401']
    result_cve_list = []
    resultpath = '../../data/NVDtest/'
    mkdir(resultpath)

writefile = 1
import time
for i,cve in enumerate(cveList):
    #if cve in result_cve_list or not cve.startswith('cve'):
    if not cve.startswith('cve') or cve in result_cve_list:
        continue
#     if i==1:
#         break
#     if i!= 0 and i%500 == 0:
#         time.sleep(30)
    
    
    url = 'https://nvd.nist.gov/vuln/detail/' + cve.upper()
    
    #print(url)
    tree = get_tree(url)
    #print(tree)
    description_path = basepath+'"]/tr/td/div/div[1]/p[1]'
    description = get_content(tree, description_path)
    while not description:
        time.sleep(10)
        #tree = get_tree(url)
        description = get_content(tree, description_path)
    #print(description)
    print(i,':',cve)
    cvss_list = []
    '''
    for i in range(1,4):
        cvss_path = basepath + '_Vuln3CvssPanel"]/div/div/span/span/a'
        cvss_vector_path = basepath + '_Vuln3CvssPanel"]/div['+str(i)+']/div[3]/span/span'
        cvss_source_path = basepath + '_Vuln3CvssPanel"]/div['+str(i)+']/div[1]/div/div[2]/span'
        
        cvss = get_content(tree, cvss_path)
        cvss_vector = get_content(tree, cvss_vector_path)
        cvss_source = get_content(tree, cvss_source_path)
        
        if cvss:
            cvss_list.append([cvss_source,cvss, cvss_vector])
            print(cvss,cvss_vector, cvss_source)
    '''
    cvss_path =  basepath + '_Vuln3CvssPanel"]/div/div/span/span/a'
    cvss_vector_path = basepath + '_Vuln3CvssPanel"]/div/div[3]/span/span'
    cvss_source_path = basepath + '_Vuln3CvssPanel"]/div/div[1]/div/div[2]/span'

    cvss = get_content(tree, cvss_path)
    cvss_vector = get_content(tree, cvss_vector_path)
    cvss_source = get_content(tree, cvss_source_path)
    for i in range(len(cvss)):
        cvss_list.append([cvss[i], cvss_vector[i], cvss_source[i]])
    cvss2_list = []
    cvss2_path =  basepath + '_Vuln2CvssPanel"]/div/div/span/span/a'
    cvss2_vector_path = basepath + '_Vuln2CvssPanel"]/div/div[3]/span/span'
    cvss2_source_path = basepath + '_Vuln2CvssPanel"]/div/div[1]/div/div[2]/span'
    cvss2 = get_content(tree, cvss2_path)
    cvss2_vector = get_content(tree, cvss2_vector_path)
    cvss2_source = get_content(tree, cvss2_source_path)
    for i in range(len(cvss2)):
        cvss2_list.append([cvss2[i], cvss2_vector[i], cvss2_source[i]])
    #print(cvss_list)
    
    cwe_list=[]
    
    cwe_id_path = basepath+'_VulnTechnicalDetailsDiv"]/table/tbody/tr/td[1]/a'
    cwe_name_path = basepath+'_VulnTechnicalDetailsDiv"]/table/tbody/tr/td[2]'
    cwe_ids = get_content(tree, cwe_id_path)
    cwe_names = get_content(tree, cwe_name_path)
    for i in range(len(cwe_ids)):
        cwe_list.append([cwe_ids[i],cwe_names[i]])
    
    cpe_list = []
   
    configure_list = []
    device_list = []
    variate = 0
    for i in range(1,100):
        
        cpe_path = '//*[@id="config-div-'+str(i)+'"]/table/tr[1]/td/b'
        cpe_entry = get_content(tree, cpe_path)
        if not cpe_entry:
            cpe_path = '//*[@id="config-div-'+str(i)+'"]/table/tr/td[1]/b'
            cpe_entry = get_content(tree, cpe_path)
            variate = 1
            if not cpe_entry:
                break
        
        for j in range(1,100):
    
            cpe_path = '//*[@id="config-div-'+str(i)+'"]/table/tr['+str(j)+']/td/b'
           
            if variate == 1:
                cpe_path = '//*[@id="config-div-'+str(i)+'"]/table/tr/td['+str(j)+']/b'
           
            cpe_entry = get_content(tree, cpe_path)
            if not cpe_entry:
                break
            if cpe_entry[0].startswith('Run'):
                device_path ='//*[@id="config-div-'+str(i)+'"]/table/tr['+str(j+1)+']/td/b'
                device, version = process_cpe(tree,device_path)
                device_list.append([device,version])
                break
            configure, version = process_cpe(tree,cpe_path)
            configure_list.append([configure,version])

               
                
                
#         device_exists = 0
#         i_path = '//*[@id="config-div-'+str(i)+'"]/table/tr[2]/td/b'
#         indicator = get_content(tree,i_path)
#         if len(indicator):
            
#             if indicator[0].startswith('Run'):
#                 device_exists = 1
#                 print(i,'Device Exists')
                
#                 configure_path  = '//*[@id="config-div-'+str(i)+'"]/table/tr[1]/td/b'
#                 configure, version = process_cpe(tree,configure_path)
#                 configure_list.append([configure,version])
               
#                 device_path = '//*[@id="config-div-'+str(i)+'"]/table/tr[3]/td/b'
#                 device, version = process_cpe(tree,device_path)
#                 device_list.append([device,version])
            
#             else:
#                 for j in range(1,100):
                    
#                     cpe_path = '//*[@id="config-div-'+str(i)+'"]/table/tr['+str(j)+']/td/b'
#                     cpe_entry = get_content(tree, cpe_path)
#                     if not cpe_entry:
#                         break
#                     configure, version = process_cpe(tree,cpe_path)
#                     configure_list.append([configure,version])
               
               
        
          
#     device_list = []
#     for i in range(1,100):
#         cpe_path = '//*[@id="config-div-'+str(i)+'"]/table/tr[3]/td/b'
#         cpe_entry = get_content(tree, cpe_path)
#         if not cpe_entry:
           
#             break
#         for item in cpe_entry:
#             if item.startswith('cpe'):
#                 cpe = CPE(item)
#                 device = cpe.get_product()[0]
#                 version = cpe.get_version()[0]
#                 device_list.append([device,version])
    #print(device_list)
    reference_path = basepath+'_VulnHyperlinksPanel"]/table/tbody/tr/td[1]/a'
    reference = get_content(tree, reference_path)
    #print(reference)
    filename = resultpath + cve + '.txt'
    if writefile == 0:
        continue
    if  description:
        valid_nvd+=1
        with open(filename, 'w') as f:
            f.write('>>>CVE\n')
            f.write(cve+'\n')
            f.write('CVE<<<\n\n') 

            f.write('>>>Description\n')
            if len(description):
                f.write(description[0] + '\n')
            else:
                print("Not found description:",cve)
            f.write('Description<<<\n\n')

            f.write('>>>CVSS_3\n')
            for item in cvss_list:
                f.write(item[0]+'||'+item[1]+'||'+item[2]+'\n')
            f.write('CVSS_3<<<\n\n') 

            f.write('>>>CVSS_2\n')
            for item in cvss2_list:
                f.write(item[0]+'||'+item[1]+'||'+item[2]+'\n')
            f.write('CVSS_2<<<\n\n') 

            f.write('>>>CWE\n')
            for item in cwe_list:
                f.write(item[0]+'||'+item[1]+'\n')
            f.write('CWE<<<\n\n') 

            f.write('>>>Device\n')
            for item in device_list:
                f.write(item[0]+'||'+item[1]+'\n')
            f.write('Device<<\n\n') 

            f.write('>>>Configure\n')
            for item in configure_list:
                f.write(item[0]+'||'+item[1]+'\n')
            f.write('Configure<<\n\n') 

            f.write('>>>Reference\n')
            for item in reference:
                f.write(item+'\n')
            f.write('Reference<<<\n')

    



33 : cve-2015-3963
34 : cve-2016-1488
63 : cve-2019-18251
81 : cve-2016-3988
92 : cve-2015-7861
106 : cve-2017-12705
107 : cve-2017-16743
108 : cve-2017-16741
117 : cve-2018-4840
118 : cve-2018-4839
128 : cve-2019-13525
136 : cve-2019-10976
137 : cve-2019-10972
148 : cve-2016-9155
158 : cve-2018-1149
159 : cve-2018-1150
181 : cve-2013-3633
182 : cve-2013-3634
189 : cve-2018-13812
190 : cve-2018-13813
234 : cve-2015-6463
239 : cve-2019-13538
252 : cve-2016-9369
259 : cve-2013-2788
261 : cve-2015-7918
278 : cve-2014-4685
304 : cve-2019-9008
323 : cve-2019-13531
324 : cve-2019-13535
331 : cve-2016-5788
343 : cve-2018-4833
349 : cve-2011-4535
354 : cve-2019-13557


In [ ]:
print(valid_nvd)